# Load Data

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel

import pickle

# load dataset
data = pd.read_csv('D:/OneDrive - 한동대학교/PROJECT/트머프로젝트/Data/BTC_train.csv')
# data = pd.read_csv('./BTC_sum_plus_nn_features.csv', nrows=30000)

# In practice, feature selection should be done after data pre-processing,
# so ideally, all the categorical variables are encoded into numbers,
# and then you can assess how deterministic they are of the target

# here for simplicity I will use only numerical variables
# select numerical columns:

numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]

In [9]:
print(data.shape) # Check the shape of NN file.

pd.set_option('display.max_columns', None)
data.head(3)

(74280, 9301)


,realized_volatility,realized_volatility_mean0,num_trades,lowest_return,highest_return,high_low_gap,trade_vol,volume_power,end_price,BB_width_w20,BB_width_w40,BB_width_w10,dv1_realized_volatility,dv5_realized_volatility_mean0,dv2_lowest_return,dv3_highest_return,realized_volatility_150,realized_volatility_mean0_150,num_trades_150,lowest_return_150,highest_return_150,high_low_gap_150,trade_vol_150,realized_volatility_300,realized_volatility_mean0_300,num_trades_300,lowest_return_300,highest_return_300,high_low_gap_300,trade_vol_300,realized_volatility_450,realized_volatility_mean0_450,num_trades_450,lowest_return_450,highest_return_450,high_low_gap_450,trade_vol_450,volume_power_450,liq_last_1,liq_last_2,liq_last_5,liq_last_10,liq_last_15,ep_liq_5,bidask_spread_0,bidask_spread_1,liq_last_1_150,liq_last_2_150,liq_last_5_150,liq_last_10_150,liq_last_15_150,bidask_spread_0_150,bidask_spread_1_150,liq_last_1_300,liq_last_2_300,liq_last_5_300,liq_last_10_300,liq_last_15_300,bidask_spread_0_300,bidask_spread_1_300,liq_last_1_450,liq_last_2_450,liq_last_5_450,liq_last_10_450,liq_last_15_450,bidask_spread_0_450,bidask_spread_1_450,realized_volatility_30s,dv4_realized_volatility_30s,tvpl,tvpl_epliq5,trade.tau,tvpl1,tvpl2,tvpl5,tvpl10,realized_volatility_nn2_bidask_spread_1_c_mean,realized_volatility_nn2_realized_volatility_c_mean,realized_volatility_nn2_bidask_spread_0_c_mean,realized_volatility_nn2_tvpl_c_mean,realized_volatility_nn2_tvpl_epliq5_c_mean,realized_volatility_nn2_high_low_gap_c_mean,realized_volatility_nn2_BB_width_w10_c_mean,realized_volatility_nn2_BB_width_w20_c_mean,realized_volatility_nn2_ep_liq_5_c_mean,realized_volatility_nn2_liq_last_5_c_mean,realized_volatility_nn2_liq_last_1_c_mean,realized_volatility_nn2_bidask_spread_1_m_p1_mean,realized_volatility_nn2_realized_volatility_m_p1_mean,realized_volatility_nn2_bidask_spread_0_m_p1_mean,realized_volatility_nn2_tvpl_m_p1_mean,realized_volatility_nn2_tvpl_epliq5_m_p1_mean,realized_volatility_nn2_high_low_gap_m_p1_mean,realized_volatility_nn2_BB_width_w10_m_p1_mean,realized_volatility_nn2_BB_width_w20_m_p1_mean,realized_volatility_nn2_ep_liq_5_m_p1_mean,realized_volatility_nn2_liq_last_5_m_p1_mean,realized_volatility_nn2_liq_last_1_m_p1_mean,realized_volatility_nn2_bidask_spread_1_m_p2_mean,realized_volatility_nn2_realized_volatility_m_p2_mean,realized_volatility_nn2_bidask_spread_0_m_p2_mean,realized_volatility_nn2_tvpl_m_p2_mean,realized_volatility_nn2_tvpl_epliq5_m_p2_mean,realized_volatility_nn2_high_low_gap_m_p2_mean,realized_volatility_nn2_BB_width_w10_m_p2_mean,realized_volatility_nn2_BB_width_w20_m_p2_mean,realized_volatility_nn2_ep_liq_5_m_p2_mean,realized_volatility_nn2_liq_last_5_m_p2_mean,realized_volatility_nn2_liq_last_1_m_p2_mean,realized_volatility_nn2_liq_last_1two_c_mean,realized_volatility_nn2_two_m_mean,realized_volatility_nn2_sev_high_nn_m_mean,realized_volatility_nn2_sev_low_nn_m_mean,realized_volatility_nn2_sev_high_abs_nn_m_mean,realized_volatility_nn2_sev_low_abs_nn_m_mean,realized_volatility_nn2_all_nn_m_p1_mean,realized_volatility_nn2_all_nn_m_p2_mean,realized_volatility_nn4_bidask_spread_1_c_mean,realized_volatility_nn4_realized_volatility_c_mean,realized_volatility_nn4_bidask_spread_0_c_mean,realized_volatility_nn4_tvpl_c_mean,realized_volatility_nn4_tvpl_epliq5_c_mean,realized_volatility_nn4_high_low_gap_c_mean,realized_volatility_nn4_BB_width_w10_c_mean,realized_volatility_nn4_BB_width_w20_c_mean,realized_volatility_nn4_ep_liq_5_c_mean,realized_volatility_nn4_liq_last_5_c_mean,realized_volatility_nn4_liq_last_1_c_mean,realized_volatility_nn4_bidask_spread_1_m_p1_mean,realized_volatility_nn4_realized_volatility_m_p1_mean,realized_volatility_nn4_bidask_spread_0_m_p1_mean,realized_volatility_nn4_tvpl_m_p1_mean,realized_volatility_nn4_tvpl_epliq5_m_p1_mean,realized_volatility_nn4_high_low_gap_m_p1_mean,realized_volatility_nn4_BB_width_w10_m_p1_mean,realized_volatility_nn4_BB_width_w20_m_p1_mean,realized_volatility_nn4_ep_liq_5_m_p1_mean,realize

# Remove Row Data to Selection

In [11]:
print("Before remove dv5 threshold : " , data.shape)

# 'dv1_realized_volatility' 열의 25%와 75% 백분위수 계산
percentile_25 = data['dv5_realized_volatility_mean0'].quantile(0.25)
percentile_75 = data['dv5_realized_volatility_mean0'].quantile(0.75)

# 0~25%와 75~100% 범위 밖의 행 제거
data_q1q4 = data[(data['dv5_realized_volatility_mean0'] <= percentile_25) | (data['dv5_realized_volatility_mean0'] >= percentile_75)]

print("After remove dv5 threshold : " , data_q1q4.shape)

Before remove dv5 threshold :  (74280, 9301)
After remove dv5 threshold :  (37140, 9301)


In [12]:
y = data_q1q4['dv5_realized_volatility_mean0']
X = data_q1q4.drop(['dv1_realized_volatility','dv2_lowest_return','dv3_highest_return','dv4_realized_volatility_30s','dv5_realized_volatility_mean0'], axis=1)

# Feature Selection (0)
## Remove Corrlative Features

In [13]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    
    for i in range(len(corr_matrix.columns)):
        if i % 10 == 0:
            print("progressed : ", (i/len(corr_matrix.columns)*100))        
        for j in range(i) :
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

# note that we reduce the correlation threshold
# to remove more features
corr_features = correlation(X.sample(n=2000, random_state=42), 0.9)
print('correlated features: ', len(set(corr_features)) )

progressed :  0.0
progressed :  0.10757314974182443
progressed :  0.21514629948364886
progressed :  0.3227194492254733
progressed :  0.4302925989672977
progressed :  0.5378657487091222
progressed :  0.6454388984509466
progressed :  0.7530120481927711
progressed :  0.8605851979345954
progressed :  0.96815834767642
progressed :  1.0757314974182444
progressed :  1.1833046471600688
progressed :  1.2908777969018932
progressed :  1.3984509466437178
progressed :  1.5060240963855422
progressed :  1.6135972461273667
progressed :  1.7211703958691909
progressed :  1.8287435456110155
progressed :  1.93631669535284
progressed :  2.0438898450946645
progressed :  2.1514629948364887
progressed :  2.2590361445783134
progressed :  2.3666092943201376
progressed :  2.474182444061962
progressed :  2.5817555938037864
progressed :  2.689328743545611
progressed :  2.7969018932874357
progressed :  2.90447504302926
progressed :  3.0120481927710845
progressed :  3.1196213425129087
progressed :  3.227194492254733

In [14]:
# Set을 List로 변환
corr_features = list(corr_features)

# List를 Pickle로 저장
with open('corr_list_2.pkl', 'wb') as f:
    pickle.dump(corr_features, f)

# 저장된 Pickle 파일을 로드하는 방법
with open('corr_list_2.pkl', 'rb') as f:
    corr_features = pickle.load(f)

print(len(corr_features))

4884


In [9]:
X.drop(labels=corr_features, axis=1, inplace=True)

In [10]:
X.shape

(15000, 3960)

# Feature Selction A

## Exhaustive Feature Selection

This exhaustive feature selection algorithm is a wrapper approach for brute-force evaluation of feature subsets; the best subset is selected by optimizing a specified performance metric given an arbitrary regressor or classifier.

In [17]:
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score, r2_score
from sklearn.model_selection import train_test_split

from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS

In [18]:
# exhaustive search

# in order to shorter search time for the demonstration
# i will ask the algorithm to try all possible 10 and 11
# feature combinations

# if you have access to a multicore or distributed computer
# system you can try more greedy searches

efs = EFS(RandomForestRegressor(n_estimators=5,
                                n_jobs=4,
                                random_state=0,
                                max_depth=2),
          min_features=1,
          max_features=2,
          scoring='r2',
          print_progress=True,
          cv=2)

efs = efs.fit(X, y)

KeyboardInterrupt: 

In [ ]:
data.columns[list(efs.best_idx_)]

# Feature Selction B

## Embedded Methods: Linear Model Coefficients

train a Linear regression model and select features with higher coefficients.

In [ ]:
# the features in the house dataset are in different scales
# so we train a scaler to scale them

scaler = StandardScaler()
scaler.fit(X)

# the LinearRegression object from sklearn is a non-regularised
# linear method. It fits by matrix multiplication and not 
# gradient descent.
# therefore we don't need to specify penalty and other parameters

sel_ = SelectFromModel(LinearRegression())

sel_.fit(scaler.transform(X), y)

# let's count the number of variables selected
selected_feat = X.columns[(sel_.get_support())]

# 저장할 파일 이름
file_name = "selected_feat_linear_coeff.pkl"

# selected_feat 변수를 pkl 파일로 저장
with open(file_name, 'wb') as file:
    pickle.dump(selected_feat, file)

print(f"{file_name} 파일이 저장되었습니다.")

print(len(selected_feat))


# and now, let's compare the  amount of selected features
# with the amount of features which coefficient is above the
# mean coefficient, to make sure we understand the output of
# sklearn

print('total features: {}'.format((X.shape[1])))

print('selected features: {}'.format(len(selected_feat)))

print(
    'features with coefficients greater than the mean coefficient: {}'.format(
        np.sum(
            np.abs(sel_.estimator_.coef_) > np.abs(
                sel_.estimator_.coef_).mean())))

In [ ]:
selected_feat

# Feature Selction C

## Embedded Methods: Tree Importance

train a Linear regression model and select features with higher coefficients.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# RandomForestRegressor 모델 학습
model = RandomForestRegressor(n_estimators=128, random_state=42, verbose=True)
model.fit(X, y)

# 특성 중요도 출력
feature_importances = model.feature_importances_
print("Feature Importances:", feature_importances)

# 특성 선택: 중요도가 높은 순서대로 특성 선택 (임계값 설정 가능)
selected_features_indices = feature_importances.argsort()[::-1]  # 중요도 내림차순 정렬
selected_features = X.columns[selected_features_indices]
print("Selected Features:", selected_features)

In [ ]:
# 저장할 파일 이름
file_name = "selected_feat_ranfo.pkl"

# selected_feat 변수를 pkl 파일로 저장
with open(file_name, 'wb') as file:
    pickle.dump(selected_features, file)

print(f"{file_name} 파일이 저장되었습니다.")

print(len(selected_features))

In [ ]:
# 리스트를 DataFrame으로 변환
data = {'Name': list(selected_features), 'Importance': sorted(list(feature_importances), reverse=True)}
df = pd.DataFrame(data)

# DataFrame 출력
df.head(50)